In this notebook, we will use time series analysis techniques on Sentinel-1 GRD images to predict floods. 

Time series analysis of Sentinel-1 Ground Range Detected (GRD) images has emerged as a powerful technique for flood prediction and monitoring. Sentinel-1 is a constellation of European Space Agency (ESA) satellites that provide high-quality Synthetic Aperture Radar (SAR) data with high temporal and spatial resolution. This makes Sentinel-1 an ideal data source for studying floods, which are highly dynamic and often occur in remote or hard-to-reach areas.

In [ ]:
# installing requirement
!pip install rasterio
!pip install datetime

In [ ]:
# importing libraries
import os
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# mounting with google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read multiple GeoTIFF files and stack them along the time dimension
def read_data(data_dir, filenames):
    data = []
    for filename in filenames:
        with rasterio.open(os.path.join(data_dir, filename)) as src:
            data.append(src.read(1))
    return np.stack(data, axis=0)

In [ ]:
# Compute intensity time-series as the mean value over time
def compute_intensity(time_series):
    return np.mean(time_series, axis=(1, 2))

In [ ]:
# Fit a linear regression model to the time-series and plot the results
def plot_linear_regression(time_series, title):
    X = np.arange(len(time_series)).reshape(-1, 1)
    y = compute_intensity(time_series)
    model = LinearRegression().fit(X, y)
    plt.plot(X, y, label='Data')
    plt.plot(X, model.predict(X), label='Linear Regression')
    plt.title(title)
    plt.legend()
    plt.savefig(f'{title}.png')
    plt.close()

In [ ]:
# Fit a linear regression model to the time-series and return the coefficient
def compute_linear_regression(time_series):
    X = np.arange(len(time_series)).reshape(-1, 1)
    y = compute_intensity(time_series)
    model = LinearRegression().fit(X, y)
    return model.coef_[0]

In [ ]:
# Write a numpy array to disk as a CSV file
def write_results(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [ ]:
def study_and_predict_flood(data_dir, before_vv_filenames, before_vh_filenames, during_vv_filenames, during_vh_filenames, after_vv_filenames, after_vh_filenames):
    # Read time-series data for before, during, and after floods
    before_time_series = read_data(data_dir, before_vv_filenames + before_vh_filenames)
    during_time_series = read_data(data_dir, during_vv_filenames + during_vh_filenames)
    after_time_series = read_data(data_dir, after_vv_filenames + after_vh_filenames)

    # Compute intensity time-series for before, during, and after floods
    before_intensity = compute_intensity(before_time_series)
    during_intensity = compute_intensity(during_time_series)
    after_intensity = compute_intensity(after_time_series)

    # Plot linear regression for before flood intensity time-series
    plot_linear_regression(before_time_series, 'Before Flood')

    # Plot linear regression for during flood intensity time-series
    plot_linear_regression(during_time_series, 'During Flood')

    # Plot linear regression for after flood intensity time-series
    plot_linear_regression(after_time_series, 'After Flood')

    # Predict likelihood of flood based on linear regression
    before_coef = compute_linear_regression(before_time_series)
    during_coef = compute_linear_regression(during_time_series)
    after_coef = compute_linear_regression(after_time_series)

    # Write results to disk
    write_results(before_intensity, 'before_intensity.csv')
    write_results(during_intensity, 'during_intensity.csv')
    write_results(after_intensity, 'after_intensity.csv')
    write_results([before_coef, during_coef, after_coef], 'linear_regression_coefficients.csv')

In [ ]:
# specification of input data files and directory path
data_dir = '/content/drive/My Drive/satellite_data/all_processed'

before_vv_filenames = ['before_flood_VV.tif']
before_vh_filenames = ['before_flood_VH.tif']
during_vh_filenames = ['during_flood_VH.tif']
during_vv_filenames = ['during_flood_VV.tif']
after_vv_filenames = ['after_flood_VV.tif']
after_vh_filenames = ['after_flood_VH.tif']

In [ ]:
study_and_predict_flood(data_dir, before_vv_filenames, before_vh_filenames, during_vv_filenames, during_vh_filenames, after_vv_filenames, after_vh_filenames)

In [ ]:
before_intensity = pd.read_csv('before_intensity.csv')
during_intensity = pd.read_csv('during_intensity.csv')
after_intensity = pd.read_csv('after_intensity.csv')
linear_regression_coefficients = pd.read_csv('linear_regression_coefficients.csv')

In [ ]:
# Plot intensity time-series
plt.plot(before_intensity, label='Before Flood(2028/06/10)')
plt.plot(during_intensity, label='During Flood(2018/07/28)')
plt.plot(after_intensity, label='After Flood(2018/10/20)')
plt.legend()
plt.show()

The intensity time series plot above provides a graphical representation of the changes in the intensity of a particular variable over time. 

Based on the given statement, the before flood line lies between during flood and after flood lines, indicating that the intensity of the variable increases before the flood, reaches its peak during the flood, and decreases after the flood. This pattern may have significant implications for the variable being measured and its effects on the surrounding environment. Understanding the changes in intensity before, during, and after the flood can be crucial in predicting the impact of floods on the ecosystem and developing effective management strategies. 

In [ ]:
# Plot linear regression coefficients
plt.plot(linear_regression_coefficients, marker='o')
plt.xticks([0, 1, 2], ['Before', 'During', 'After'])
plt.ylabel('Linear Regression Coefficient')
plt.show()

The linear coefficients before, during, and after a flood indicate the magnitude and direction of changes in various factors such as water level, flow rate, sediment transport, and water quality. 

Based on the given statement, the linear coefficient before the flood is greater than during the flood, which suggests that the rate of change in these factors before the flood is higher than during the flood. 
The linear coefficients before, during, and after a flood indicate the magnitude and direction of changes in various factors such as water level, flow rate, sediment transport, and water quality. 
Based on the given statement, the linear coefficient before the flood is greater than during the flood, which suggests that the rate of change in these factors before the flood is higher than during the flood.